In [1]:
import gpxpy
import numpy as np
import pandas as pd
from kompy import KomootConnector
import os
connector = KomootConnector(
    email=os.environ['KOMOOT_EMAIL'],
    password=os.environ['KOMOOT_PASSWORD'],
)
gpx_track = connector.get_tour_by_id(tour_identifier='1378844177', object_type='gpx')


In [2]:
gpx_track

GPX(tracks=[GPXTrack(name='W02D2-Recovery Run', segments=[GPXTrackSegment(points=[...])])])

In [3]:
from gpxpy.gpx import GPXTrackPoint
import math
from geopy import distance

def calculate_initial_compass_bearing(first_point: GPXTrackPoint, second_point: GPXTrackPoint):
    """
    Calculates the bearing between two points.

    The formula used to calculate the bearing is:
        θ = atan2(sin(Δlong).cos(lat2), cos(lat1).sin(lat2) - sin(lat1).cos(lat2).cos(Δlong))

    :param first_point: The first point
    :param second_point: The second point
    :return: The bearing between the two points in degrees
    """

    lat1 = math.radians(first_point.latitude)
    lat2 = math.radians(second_point.latitude)
    diffLong = math.radians(second_point.longitude - first_point.longitude)

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Normalize the initial bearing
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

def calculate_turn_angle(bearing1, bearing2):
    """
    Calculate the angle of turn between two bearings.
    """
    angle = bearing2 - bearing1
    
    angle = (angle + 180) % 360 - 180

    return angle




def load_gpx_todf(gpx_data):
    gpx_points = []
    for track in gpx_data.tracks:
        for segment in track.segments:
            for point in segment.points:
                gpx_points.append(
                    {
                        'gpx_point': point,
                        "latitude": point.latitude,
                        "longitude": point.longitude,
                        "elevation": point.elevation,
                        "time": point.time,
                        "distance": distance.geodesic(
                            (point.latitude, point.longitude),
                            (gpx_points[-1]['latitude'],gpx_points[-1]['longitude']),
                        ).m if len(gpx_points) > 0 else 0,
                        'time_diff_s': (point.time - gpx_points[-1]['gpx_point'].time).total_seconds() if len(gpx_points) > 0 else 0,
                        'elevation_diff': point.elevation - gpx_points[-1]['gpx_point'].elevation if len(gpx_points) > 0 else 0,
                        'bearing': calculate_initial_compass_bearing(
                            first_point=point,
                            second_point=gpx_points[-1]['gpx_point'],
                        ) if len(gpx_points) > 0 else 0,
                        'turn_angle': calculate_turn_angle(
                            bearing1=calculate_initial_compass_bearing(
                                first_point=gpx_points[-1]['gpx_point'],
                                second_point=gpx_points[-2]['gpx_point'],
                            ),
                            bearing2=calculate_initial_compass_bearing(
                                first_point=point,
                                second_point=gpx_points[-1]['gpx_point'],
                            ),
                        ) if len(gpx_points) > 1 else 0,
                    }
                )
    return pd.DataFrame(gpx_points)

In [58]:
gpx_df = load_gpx_todf(gpx_track)

In [59]:
gpx_df

,gpx_point,latitude,longitude,elevation,time,distance,time_diff_s,elevation_diff,bearing,turn_angle
0,"[trkpt:45.736284,7.315084@550.838341@2023-11-2...",45.736284,7.315084,550.838341,2023-11-20 17:02:16+00:00,0.000000,0.0,0.0,0.000000,0.000000
1,"[trkpt:45.73628,7.315069@550.838341@2023-11-20...",45.736280,7.315069,550.838341,2023-11-20 17:02:17+00:00,1.249245,1.0,0.0,69.089899,0.000000
2,"[trkpt:45.736266,7.315043@550.838341@2023-11-2...",45.736266,7.315043,550.838341,2023-11-20 17:02:18+00:00,2.552687,1.0,0.0,52.350637,-16.739262
3,"[trkpt:45.73625,7.315014@550.838341@2023-11-20...",45.736250,7.315014,550.838341,2023-11-20 17:02:19+00:00,2.873486,1.0,0.0,51.674378,-0.676260
4,"[trkpt:45.736222,7.31498@550.838341@2023-11-20...",45.736222,7.314980,550.838341,2023-11-20 17:02:20+00:00,4.085058,1.0,0.0,40.282128,-11.392250
...,...,...,...,...,...,...,...,...,...,...
3704,"[trkpt:45.735716,7.315114@551.559318@2023-11-2...",45.735716,7.315114,551.559318,2023-11-20 18:04:00+00:00,1.336023,1.0,0.0,183.328807,-20.976426
3705,"[trkpt:45.735729,7.315104@551.559318@2023-11-2...",45.735729,7.315104,551.559318,2023-11-20 18:04:01+00:00,1.641191,1.0,0.0,151.768682,-31.560125
3706,"[trkpt:45.735752,7.315098@551.559318@2023-11-2...",45.735752,7.315098,551.559318,2023-11-20 18:04:02+00:00,2.598665,1.0,0.0,169.680698,17.912016
3707,"[trkpt:45.735774,7.315096@551.559318@2023-11-2...",45.735774,7.315096,551.559318,2023-11-20 18:04:03+00:00,2.450165,1.0,0.0,176.369353,6.688655


In [119]:
from scipy import signal
import numpy as np

gpx_df['turn_moving_cumsum'] = gpx_df['turn_angle'].rolling(window=20,min_periods=1).average()



AttributeError: 'Rolling' object has no attribute 'average'

In [120]:
from scipy.signal import argrelextrema


def identify_local_minmax(df, input_column_name, output_column_name='relative_extrema', order=None):
    # Initialize a new column 'turn' with NaN values
    df[output_column_name] = 0

    # Identify indices of local minima and maxima
    if not order:
        minima_indices = argrelextrema(df[input_column_name].values, np.less_equal)[0]
        maxima_indices = argrelextrema(df[input_column_name].values, np.greater_equal)[0]
    else:
        minima_indices = argrelextrema(df[input_column_name].values, np.less_equal, order=order)[0]
        maxima_indices = argrelextrema(df[input_column_name].values, np.greater_equal, order=order)[0]

    # Mark local minima and maxima in the 'turn' column
    df.loc[minima_indices, output_column_name] = df.loc[minima_indices, input_column_name]
    df.loc[maxima_indices, output_column_name] = df.loc[maxima_indices, input_column_name]

    return df

In [121]:
gpx_df = identify_local_minmax(gpx_df, input_column_name='turn_moving_cumsum', output_column_name='turn', order=100)
gpx_df = identify_local_minmax(gpx_df, input_column_name='elevation', output_column_name='elevation_extrema')

/var/folders/ks/7zd0y9gn3b1f5y7zw35zrm9r0000gq/T/ipykernel_97219/1385703926.py:17: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ -84.22039197 -111.70695554  -78.31073616  -48.47144929  -81.36126527
 -102.40029006  -86.21600521  -98.46727896  -81.66401473  -84.53194498
  -16.36400635  -11.39463239  -74.33200373  -17.95904754  -26.01416386
 -397.66256643  -78.26848926  -67.41391043]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/var/folders/ks/7zd0y9gn3b1f5y7zw35zrm9r0000gq/T/ipykernel_97219/1385703926.py:17: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[550.838341 550.838341 550.838341 550.838341 550.838341 550.838341
 550.838341 550.838341 550.838341 550.838341 550.838341 550.838341
 550.838341 550.838341 550.838341 550.838341 550.838341 550.838341
 550.838341 550.838341 550.838341 550.838341 550.

In [122]:
gpx_df

,gpx_point,latitude,longitude,elevation,time,distance,time_diff_s,elevation_diff,bearing,turn_angle,turn_moving_cumsum,turn,elevation_extrema
0,"[trkpt:45.736284,7.315084@550.838341@2023-11-2...",45.736284,7.315084,550.838341,2023-11-20 17:02:16+00:00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,550.838341
1,"[trkpt:45.73628,7.315069@550.838341@2023-11-20...",45.736280,7.315069,550.838341,2023-11-20 17:02:17+00:00,1.249245,1.0,0.0,69.089899,0.000000,0.000000,0.0,550.838341
2,"[trkpt:45.736266,7.315043@550.838341@2023-11-2...",45.736266,7.315043,550.838341,2023-11-20 17:02:18+00:00,2.552687,1.0,0.0,52.350637,-16.739262,-9.302996,0.0,550.838341
3,"[trkpt:45.73625,7.315014@550.838341@2023-11-20...",45.736250,7.315014,550.838341,2023-11-20 17:02:19+00:00,2.873486,1.0,0.0,51.674378,-0.676260,-10.835175,0.0,550.838341
4,"[trkpt:45.736222,7.31498@550.838341@2023-11-20...",45.736222,7.314980,550.838341,2023-11-20 17:02:20+00:00,4.085058,1.0,0.0,40.282128,-11.392250,-18.361222,0.0,550.838341
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3704,"[trkpt:45.735716,7.315114@551.559318@2023-11-2...",45.735716,7.315114,551.559318,2023-11-20 18:04:00+00:00,1.336023,1.0,0.0,183.328807,-20.976426,-8.098927,0.0,551.559318
3705,"[trkpt:45.735729,7.315104@551.559318@2023-11-2...",45.735729,7.315104,551.559318,2023-11-20 18:04:01+00:00,1.641191,1.0,0.0,151.768682,-31.560125,-14.905413,0.0,551.559318
3706,"[trkpt:45.735752,7.315098@551.559318@2023-11-2...",45.735752,7.315098,551.559318,2023-11-20 18:04:02+00:00,2.598665,1.0,0.0,169.680698,17.912016,-8.704907,0.0,551.559318
3707,"[trkpt:45.735774,7.315096@551.559318@2023-11-2...",45.735774,7.315096,551.559318,2023-11-20 18:04:03+00:00,2.450165,1.0,0.0,176.369353,6.688655,-18.564719,0.0,551.559318


In [123]:
import plotly.express as px


In [124]:
import plotly.graph_objects as go

scatter = go.Scatter(
    x=gpx_df['time'],
    y=gpx_df['elevation'],
    mode='lines',
)

trace2 = go.Scatter(
    x=gpx_df[gpx_df['elevation_extrema'] != 0]['time'],
    y=gpx_df[gpx_df['elevation_extrema'] != 0]['elevation_extrema'],
    mode='markers',
)

fig = go.Figure(data=[scatter, trace2])
fig.show()

In [125]:
scatter = go.Scatter(
    x=gpx_df['time'],
    y=gpx_df['turn_moving_cumsum'],
    mode='lines',
)

trace2 = go.Scatter(
    x=gpx_df[gpx_df['turn'] != 0]['time'],
    y=gpx_df[gpx_df['turn'] != 0]['turn'],
    mode='markers',
)

fig = go.Figure(data=[scatter, trace2])
fig.show()


In [126]:

fig = px.scatter_mapbox(
    gpx_df[:3000],
    lat="latitude",
    lon="longitude",
    hover_name="time_diff_s",
    hover_data=["turn_moving_cumsum", "turn_angle"],
    color="turn",
    zoom=12,
    width=1200,
    height=700,
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()